In [1]:
import shutil
import subprocess
import glob
from tqdm import tqdm
import numpy as np
import cv2
import os
from keras.applications.vgg16 import VGG16
from keras.models import Model 
import fire
from elapsedtimer import ElapsedTimer
import cv2

Using TensorFlow backend.


In [2]:
video_dest = 'C:\\Users\\cst2978\\OneDrive - The Home Depot\\Desktop\\AI\\qsb\\Project\\YouTubeClips'
feat_dir = 'C:\\Users\\cst2978\\OneDrive - The Home Depot\\Desktop\\AI\\qsb\\Project\\features'
temp_dest = 'C:\\Users\\cst2978\\OneDrive - The Home Depot\\Desktop\\AI\\qsb\\Project\\temp\\'
img_dim = 224
channels=3
batch_size=128
batch_cnn =128
frames_step=80

os.chdir('C:/Users/cst2978/OneDrive - The Home Depot/Desktop/AI/qsb/Project')

In [3]:
def video_to_frames(video):
    os.chdir('C:/Users/cst2978/OneDrive - The Home Depot/Desktop/AI/qsb/Project/YouTubeClips')
    if not os.path.isdir(temp_dest):
        os.mkdir(temp_dest)
    print (video)
    vidcap = cv2.VideoCapture(video)
    print (vidcap)
    def getFrame(sec):
        vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        hasFrames,image = vidcap.read() 
        if hasFrames:
            cv2.imwrite(temp_dest+"image"+str(count)+".jpg", image)     # save frame as JPG file
        return hasFrames
    sec = 0
    frameRate = 0.5 #//it will capture image in each 0.5 second
    count=1
    success = getFrame(sec)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)

In [4]:
def model_cnn_load():
         model = VGG16(weights = "imagenet", include_top=True,input_shape = 
                                  (img_dim,img_dim,channels))
         out = model.layers[-2].output
         model_final = Model(input=model.input,output=out)
         return model_final

In [5]:
def load_image(path):
        img = cv2.imread(path)
        img = cv2.resize(img,(img_dim,img_dim))
        return img 

In [6]:
#Loading VGG16 model
model = model_cnn_load()

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\cst2978\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  """


In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
def extract_feats_pretrained_cnn():
        
    model = model_cnn_load()
    print('Model loaded') 
    if not os.path.isdir(feat_dir):
        os.mkdir(feat_dir)
        #print("save video feats to %s" % (dir_feat))
    video_list = glob.glob(os.path.join(video_dest, '*.avi'))
        #print video_list 
        
    for i in range (0,len(video_list)-1):
        video = video_list[i]
        #print (video)    
        video_id = video.split("YouTubeClips")[-1]
        video_id = video_id[1:]
        print(f'Processing video {video_id}')
        os.chdir('C:/Users/cst2978/OneDrive - The Home Depot/Desktop/AI/qsb/Project')
        video_to_frames(video_id)
        os.chdir('C:/Users/cst2978/OneDrive - The Home Depot/Desktop/AI/qsb/Project')
        image_list = sorted(glob.glob(os.path.join(temp_dest, '*.jpg')))
        #print (image_list)
        samples = np.round(np.linspace(0, len(image_list) - 1,frames_step))
        #print (samples)
        image_list = [image_list[int(sample)] for sample in samples]
        images = np.zeros((len(image_list),img_dim,img_dim,channels))
        for i in range(len(image_list)):
            img = load_image(image_list[i])
            images[i] = img
        images = np.array(images)
        fc_feats = model.predict(images,batch_size=batch_cnn)
        img_feats = np.array(fc_feats)
        outfile = os.path.join(feat_dir, video_id + '.npy')
        np.save(outfile, img_feats)
        # cleanup
        shutil.rmtree(temp_dest)

In [ ]:
extract_feats_pretrained_cnn()

C:\Users\cst2978\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`
  """


Model loaded
Processing video -4wsuPCjDBc_5_15.avi
-4wsuPCjDBc_5_15.avi
<VideoCapture 000002090993FB70>
Processing video -7KMZQEsJW4_205_208.avi
-7KMZQEsJW4_205_208.avi
<VideoCapture 00000209010EF470>
Processing video -8y1Q0rA3n8_108_115.avi
-8y1Q0rA3n8_108_115.avi
<VideoCapture 000002090993FA70>
Processing video -8y1Q0rA3n8_95_102.avi
-8y1Q0rA3n8_95_102.avi
<VideoCapture 00000209010EF470>
Processing video -9CUm-2cui8_39_44.avi
-9CUm-2cui8_39_44.avi
<VideoCapture 000002090993FA70>
Processing video -AwoiGR6c8M_10_14.avi
-AwoiGR6c8M_10_14.avi
<VideoCapture 000002090993FF70>
Processing video -bjOB4zS0uE_100_105.avi
-bjOB4zS0uE_100_105.avi
<VideoCapture 00000209010EF470>
Processing video -Cv5LsqKUXc_17_25.avi
-Cv5LsqKUXc_17_25.avi
<VideoCapture 00000209010EF470>
Processing video -Cv5LsqKUXc_71_76.avi
-Cv5LsqKUXc_71_76.avi
<VideoCapture 000002090993FB70>
Processing video -DKuLXYoY3g_14_20.avi
-DKuLXYoY3g_14_20.avi
<VideoCapture 000002090993FA70>
Processing video -dm-ds5rRaM_44_52.avi
-dm-ds